In [ ]:
from langchain_core.tools import tool
import requests

Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
Found existing installation: langchain-core 0.3.83
Uninstalling langchain-core-0.3.83:
  Successfully uninstalled langchain-core-0.3.83
Found existing installation: langchain-openai 0.3.35
Uninstalling langchain-openai-0.3.35:
  Successfully uninstalled langchain-openai-0.3.35
Found existing installation: langchain-text-splitters 0.3.11
Uninstalling langchain-text-splitters-0.3.11:
  Successfully uninstalled langchain-text-splitters-0.3.11
Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_openai-0.3.35-py3-none-any.whl.metadata (2.4 kB)
  Using cached langchain_core-0.3.83-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
Using cached langchain-0.3.27-py3-none-any.whl (1.0 MB)
Using 

/Users/alp/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'create_agent' from 'langchain.agents' (/Users/alp/Library/Python/3.9/lib/python/site-packages/langchain/agents/__init__.py)

In [25]:
BASE_URL = "https://hacker-news.firebaseio.com/v0/"

def get_data(endpoint): 
    URL = f"{BASE_URL}{endpoint}.json"
    response = requests.get(URL, timeout=10)
    return response.json()

@tool
def get_top_story_ids(limit=10):
    """
    Get the top {limit} story ids from Hacker News
    """
    return get_data("topstories")[:limit]

@tool
def get_story(story_id):
    """
    Get the data of any given post on Hacker News
    Data includes title, url, score, by, and time
    """
    return get_data(f"item/{story_id}")

In [14]:
top_ids = get_top_story_ids(limit=10)
for idx, story_id in enumerate(top_ids, start=1):
    item = get_story(story_id)
    title = item.get("title", "<no title>")
    url = item.get("url", f"https://news.ycombinator.com/item?id={story_id}")
    score = item.get("score", 0)
    by = item.get("by", "unknown")
    print(f"{idx}. {title} ({score} points by {by})")
    print(f"{url}")

1. East Germany balloon escape (367 points by robertvc)
https://en.wikipedia.org/wiki/East_Germany_balloon_escape
2. Cloudflare acquires Astro (751 points by todotask2)
https://astro.build/blog/joining-cloudflare/
3. FLUX.2 [Klein]: Towards Interactive Visual Intelligence (63 points by GaggiX)
https://bfl.ai/blog/flux2-klein-towards-interactive-visual-intelligence
4. 6-Day and IP Address Certificates Are Generally Available (363 points by jaas)
https://letsencrypt.org/2026/01/15/6day-and-ip-general-availability
5. LLM Structured Outputs Handbook (155 points by vitaelabitur)
https://nanonets.com/cookbooks/structured-llm-outputs
6. Cursor's latest “browser experiment” implied success without evidence (442 points by embedding-shape)
https://embedding-shapes.github.io/cursor-implied-success-without-evidence/
7. Michelangelo's first painting, created when he was 12 or 13 (324 points by bookofjoe)
https://www.openculture.com/2026/01/discover-michelangelos-first-painting.html
8. Releasing rai